In [3]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [12]:
def fork_archive(repo_name, repo_path):
    import pandas as pd
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT forked_from, repo_archived from repo
            WHERE repo_name = {repo_name}
            AND repo_path = {repo_path}
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    forked = repo_df.forked_from[0]
    archived = repo_df.repo_archived[0]
    print(archived)
    is_archived = 'ERROR'
    
    if forked != 'Parent not available':
        is_forked = True
    else:
        is_forked = False
    
    if archived == 1:
       is_archived = True
    elif archived == 0:
       is_archived = False

    print(str(is_forked), str(is_archived))
    return is_forked, is_archived
    
#repo_name = "'salt'"
repo_path = "'github.com/saltstack/'"
repo_name = "'salt-testing'"
#repo_path = "'github.com/concourse/'"
repo_name = "'rabbithub'"
repo_path = "'github.com/rabbitmq/'"
is_fork, is_archived = fork_archive(repo_name, repo_path)

0
True False


In [7]:
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT repo_path, repo_name, forked_from from repo
            where forked_from NOT LIKE '%%Parent%%'
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    repo_df

,repo_path,repo_name,forked_from
0,github.com/platform-acceleration-lab/,cf-resource,cloudfoundry-community/cf-resource
1,github.com/pivotal-education/,pcf-route-service-code,nebhale/route-service-example
2,github.com/Pivotal-sg/,laptop,thoughtbot/laptop
3,github.com/spring-attic/,spring-data-keyvalue-examples,spring-projects/spring-data-keyvalue-examples
4,github.com/spring-cloud-samples/,sleuth-documentation-apps,marcingrzejszczak/sleuth-documentation-apps
...,...,...,...
644,github.com/carbonblack/,react-toggle-button,gdowens/react-toggle-button
645,github.com/vmware/,FireBreath,firebreath/FireBreath
646,github.com/pivotal/,harbor,goharbor/harbor
647,github.com/carbonblack/,conan,conan-io/conan


In [4]:
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT repo_path, repo_name, repo_archived from repo
            where repo_archived = 1
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    repo_df

,repo_path,repo_name,repo_archived
0,github.com/bitnami/,oraclelinux-extras-base,1
1,github.com/pivotalservices/,tile-config-generator,1
2,github.com/Spring-projects/,spring-sync,1
3,github.com/SteeltoeOSS/,steeltoe-site-ci,1
4,github.com/rabbitmq/,rabbitmq-ci-fixture,1
...,...,...,...
177,github.com/SteeltoeOSS/,Logging,1
178,github.com/pivotalservices/,reveal.js,1
179,github.com/concourse/,archive-resource,1
180,github.com/rabbitmq/,rabbitmq-ha,1


In [5]:
    repo_df = pd.DataFrame()
    repo_df_query = f"""
            SELECT forked_from, repo_archived from repo
            where forked_from NOT LIKE '%%Parent%%'
            """
    repo_df = pd.read_sql_query(repo_df_query, con=engine)
    repo_df

,forked_from,repo_archived
0,cloudfoundry-incubator/apache-brooklyn-service...,0
1,pivotal-cf/terraforming-gcp,0
2,mattes/migrate,0
3,moby/hyperkit,0
4,cdavisafc/gemfire-bosh-release,0
...,...,...
613,pivotal-cf/docs-pcf-gsg,0
614,mycila/license-maven-plugin,0
615,tammersaleh/pivotal-blog,0
616,kingfadzi/cf-node-mongo,0
